In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/prometeo23-kaggle/train_absa.csv
/kaggle/input/prometeo23-kaggle/sample.csv
/kaggle/input/prometeo23-kaggle/test_absa.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/prometeo23-kaggle/train_absa.csv')
test_df = pd.read_csv('/kaggle/input/prometeo23-kaggle/test_absa.csv')
sample_df= pd.read_csv('/kaggle/input/prometeo23-kaggle/test_absa.csv')


In [3]:
train_df.head()

,text,aspect,label
0,can you check whether its cancelled completely?,cancelled,1
1,cannot rely on both milk delivery and grocery ...,Milk,0
2,"I get no notification, however the app is real...",notification,0
3,"Love this app, but would love it even more if ...",view,1
4,it does not let me load a clip on the scene,load,0


In [ ]:
# test_df.head()

In [ ]:
# sample_df.head()

In [4]:
train_df = train_df.dropna()

In [5]:
X=train_df.drop('label',axis=1)

In [6]:
Y=train_df['label']

In [10]:
list_set = set(Y)
# convert the set to the list
unique_list = (list(list_set))
for x in unique_list:
    print (x)

0
1
2


In [ ]:
print(Y.value_counts())

In [ ]:
X.info()

In [ ]:
len(Y)

In [ ]:
X.shape

In [ ]:
Y.shape

In [8]:
import tensorflow as tf

In [9]:
from keras.layers import Embedding
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM,Dropout,Dense,Bidirectional

In [10]:
### vocabulary Size
voc_size=5000

In [11]:
message = X.copy()

In [12]:
# message['text'][1]

'cannot rely on both milk delivery and grocery items.'

In [ ]:
%pip install transformer

In [ ]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [13]:
message.reset_index(inplace=True)

In [14]:
import nltk
import re ### Replacing Words
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
### Data Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
corpus_aspect = []
for i in range(0,len(message)):
#     print(i)
    review = re.sub('[^a-zA-Z]',' ',message['text'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    review_1 = re.sub('[^a-zA-Z]',' ',message['aspect'][i])
    review_1 = review_1.lower()
    review_1 = review_1.split()

    review_1 = [ps.stem(word) for word in review_1]
    review_1 = ' '.join(review_1)
    corpus_aspect.append(review_1)

In [ ]:
# corpus

In [19]:
onehot_rep = [one_hot(words,voc_size) for words in corpus]
# onehot_rep
onehot_aspect = [one_hot(words,voc_size) for words in corpus_aspect]

In [23]:
sent_length = 50
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
# print(embedded_docs)
sent_length_aspect = 15
embedded_docs_aspect = pad_sequences(onehot_aspect,padding='pre',maxlen=sent_length_aspect)
print(embedded_docs_aspect)

[[   0    0    0 ...    0    0 4383]
 [   0    0    0 ...    0    0 4373]
 [   0    0    0 ...    0    0  742]
 ...
 [   0    0    0 ...    0    0 2605]
 [   0    0    0 ...    0 1793  437]
 [   0    0    0 ...    0    0 2561]]


In [ ]:
# embedded_docs[0]

In [24]:
embedding_vector_features=73
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
# model.add(Dropout(0.3))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

2023-01-19 17:06:52.812103: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),Y.shape

In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(Y)

In [ ]:
X_final.shape,y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [ ]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

In [ ]:
y_pred = (model.predict(X_test)>0.5).astype('int32')

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
Y_test

In [ ]:
confusion_matrix(Y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,y_pred))